In [5]:
from lightweight import lightweight

lightweight("Write a poem about dogs")

Loading model facebook/opt-125m on mps
Response: dogs and give it to your kids. Do this on Valentine's Day, and then take it to your


"dogs and give it to your kids. Do this on Valentine's Day, and then take it to your"

In [ ]:
lightweight("Write a poem about dogs",model_name="facebook/opt-350m")
lightweight("Write a poem about dogs",model_name="microsoft/phi-1_5")
lightweight("Write a poem about dogs",model_name="TinyLlama/TinyLlama-1.1B-Chat-v0.6")

Loading model facebook/opt-350m on mps
Response: dogs, but for a dog.</s>


'dogs, but for a dog.</s>'

Loading model microsoft/phi-1_5 on mps
Response: using the rhyme scheme of ABAB.
Answer: I have a furry friend
Her


'using the rhyme scheme of ABAB.\nAnswer: I have a furry friend\nHer'

Loading model TinyLlama/TinyLlama-1.1B-Chat-v0.6 on mps
Response: dogs that describes the emotional and physical impact that dogs have on their owners. Use descriptive language and vivid imagery to convey the joy and love that dogs bring to their owners. Consider incorporating metaphors or similes to evoke strong emotions. Your poem should be at least 15 lines long and aim to captivate the readers with your artistic and emotional expression of love and loyalty.</s>


'dogs that describes the emotional and physical impact that dogs have on their owners. Use descriptive language and vivid imagery to convey the joy and love that dogs bring to their owners. Consider incorporating metaphors or similes to evoke strong emotions. Your poem should be at least 15 lines long and aim to captivate the readers with your artistic and emotional expression of love and loyalty.</s>'